In [1]:
import pyarrow.parquet as pq

In [2]:
import numpy as np
import pandas as pd
import pyarrow as pa

In [127]:
df = pd.DataFrame({'one': [-1, np.nan, 2.5],
                   'two': ['foo', 'bar', 'baz'],
                   'three': [True, False, True]},
                   )
df

,one,two,three
0,-1.0,foo,True
1,NaN,bar,False
2,2.5,baz,True


In [128]:
table = pa.Table.from_pandas(df)
table

pyarrow.Table
one: double
two: string
three: bool
----
one: [[-1,null,2.5]]
two: [["foo","bar","baz"]]
three: [[true,false,true]]

In [129]:
pq.write_table(table, 'example.parquet')

In [130]:
table2 = pq.read_table('example.parquet')

In [131]:
table2.to_pandas()

,one,two,three
0,-1.0,foo,True
1,NaN,bar,False
2,2.5,baz,True


In [132]:
pq.read_table('example.parquet', columns=['one', 'three']).to_pandas()

,one,three
0,-1.0,True
1,NaN,False
2,2.5,True


In [133]:
pq.read_table('example.parquet', columns=['one', 'three'], filters=[('one', '==', 2.5)]).to_pandas()

,one,three
0,2.5,True


#### if we use `write_to_dataset` we can automatically append
append seemed difficult with `write_table` and `pq.ParquetWriter`

In [144]:
dfNew = pd.DataFrame({'one':[3.0], 'two': ['B'], 'three': [True]})
dfNew

,one,two,three
0,3.0,B,True


In [145]:
df.append(dfNew)

,one,two,three
0,-1.0,foo,True
1,NaN,bar,False
2,2.5,baz,True
0,3.0,B,True


In [146]:
table.schema.remove_metadata()

one: double
two: string
three: bool

In [147]:
newTable = pa.Table.from_pandas(dfNew).replace_schema_metadata(None)

In [148]:
newTable.schema

one: double
two: string
three: bool

In [152]:
pq.write_to_dataset(newTable, root_path='sample.parquet')

In [153]:
pq.write_to_dataset(pa.Table.from_pandas(df), root_path='sample.parquet')

In [155]:
pq.read_table('sample.parquet').to_pandas()

,one,two,three
0,3.0,B,True
1,-1.0,foo,True
2,NaN,bar,False
3,2.5,baz,True


In [156]:
pq.read_table('sample.parquet', columns=['one', 'three'], filters=[('one', '==', 2.5)]).to_pandas()

,one,three
0,2.5,True


#### bad way to append:

```
pqwriter = pq.ParquetWriter(
    'example.parquet', 
    table.schema.remove_metadata(),
    #table.replace_schema_metadata(None),
)
#pqwriter.write_table(df)
```

```
pqwriter.write_table(
    pa.Table.from_pandas(dfNew).replace_schema_metadata(None), 
    'example.parquet',

    #schema=table.schema.remove_metadata()
)
```

```
# close the parquet writer
if pqwriter:
    pqwriter.close()
```

---

### merge the data into one dataframe
once we pull the various data set from disk, we need to merge the dataframes into one, filling in values where the dates do not match up

In [23]:
import datetime as dt

In [28]:
x = dt.datetime.now()
x.ctime()

'Tue Feb 15 21:55:33 2022'

In [40]:
str(dt.datetime.utcnow()-dt.timedelta(hours=1))

'2022-02-16 03:58:30.112551'

In [43]:
df1 = pd.DataFrame({
    'dt': [
        str(dt.datetime.utcnow()-dt.timedelta(hours=2)), 
        str(dt.datetime.utcnow()-dt.timedelta(hours=1)), 
        str(dt.datetime.utcnow())],
    'value': [1, 2, 3],})
df1

,dt,value
0,2022-02-16 03:00:05.915053,1
1,2022-02-16 04:00:05.915053,2
2,2022-02-16 05:00:05.915053,3


In [45]:
df2 = pd.DataFrame({
    'dt': [
        str(dt.datetime.utcnow()-dt.timedelta(minutes=130)), 
        str(dt.datetime.utcnow()-dt.timedelta(minutes=120)), 
        str(dt.datetime.utcnow()-dt.timedelta(minutes=110)), 
        str(dt.datetime.utcnow()-dt.timedelta(minutes=100)), 
        str(dt.datetime.utcnow()-dt.timedelta(minutes=90)), 
        str(dt.datetime.utcnow()-dt.timedelta(minutes=80)), 
        str(dt.datetime.utcnow()-dt.timedelta(minutes=70)), 
        str(dt.datetime.utcnow()-dt.timedelta(minutes=60)), 
        str(dt.datetime.utcnow()-dt.timedelta(minutes=50)), 
        str(dt.datetime.utcnow()-dt.timedelta(minutes=40)), 
        str(dt.datetime.utcnow()-dt.timedelta(minutes=30)), 
        str(dt.datetime.utcnow()-dt.timedelta(minutes=20)), 
        str(dt.datetime.utcnow()-dt.timedelta(minutes=10)), 
        str(dt.datetime.utcnow())],
    'value': [
        130,
        120,
        110,
        100,
        90,
        80,
        70,
        60,
        50,
        40,
        30,
        20,
        10,
        0]})
df2

,dt,value
0,2022-02-16 02:52:45.794120,130
1,2022-02-16 03:02:45.794120,120
2,2022-02-16 03:12:45.794120,110
3,2022-02-16 03:22:45.794120,100
4,2022-02-16 03:32:45.794120,90
5,2022-02-16 03:42:45.794120,80
6,2022-02-16 03:52:45.794120,70
7,2022-02-16 04:02:45.794120,60
8,2022-02-16 04:12:45.794120,50
9,2022-02-16 04:22:45.794120,40


In [55]:
df1['dt'] = pd.to_datetime(df1['dt'])
df2['dt'] = pd.to_datetime(df2['dt'])

df3 = pd.merge(df2, df1, how='outer', on='dt')
df3

,dt,value_x,value_y
0,2022-02-16 02:52:45.794120,130.0,NaN
1,2022-02-16 03:02:45.794120,120.0,NaN
2,2022-02-16 03:12:45.794120,110.0,NaN
3,2022-02-16 03:22:45.794120,100.0,NaN
4,2022-02-16 03:32:45.794120,90.0,NaN
5,2022-02-16 03:42:45.794120,80.0,NaN
6,2022-02-16 03:52:45.794120,70.0,NaN
7,2022-02-16 04:02:45.794120,60.0,NaN
8,2022-02-16 04:12:45.794120,50.0,NaN
9,2022-02-16 04:22:45.794120,40.0,NaN


In [63]:
df3 = df3.sort_values('dt')
df3 = df3.reset_index(drop=True)
df3

,dt,value_x,value_y
0,2022-02-16 02:52:45.794120,130.0,NaN
1,2022-02-16 03:00:05.915053,NaN,1.0
2,2022-02-16 03:02:45.794120,120.0,NaN
3,2022-02-16 03:12:45.794120,110.0,NaN
4,2022-02-16 03:22:45.794120,100.0,NaN
5,2022-02-16 03:32:45.794120,90.0,NaN
6,2022-02-16 03:42:45.794120,80.0,NaN
7,2022-02-16 03:52:45.794120,70.0,NaN
8,2022-02-16 04:00:05.915053,NaN,2.0
9,2022-02-16 04:02:45.794120,60.0,NaN


In [157]:
df3 = df3.fillna(method='ffill')
df3

,dt,value_x,value_y
0,2022-02-16 02:52:45.794120,130.0,NaN
1,2022-02-16 03:00:05.915053,130.0,1.0
2,2022-02-16 03:02:45.794120,120.0,1.0
3,2022-02-16 03:12:45.794120,110.0,1.0
4,2022-02-16 03:22:45.794120,100.0,1.0
5,2022-02-16 03:32:45.794120,90.0,1.0
6,2022-02-16 03:42:45.794120,80.0,1.0
7,2022-02-16 03:52:45.794120,70.0,1.0
8,2022-02-16 04:00:05.915053,70.0,2.0
9,2022-02-16 04:02:45.794120,60.0,2.0


In [158]:
df3 = df3.fillna(method='bfill')
df3

,dt,value_x,value_y
0,2022-02-16 02:52:45.794120,130.0,1.0
1,2022-02-16 03:00:05.915053,130.0,1.0
2,2022-02-16 03:02:45.794120,120.0,1.0
3,2022-02-16 03:12:45.794120,110.0,1.0
4,2022-02-16 03:22:45.794120,100.0,1.0
5,2022-02-16 03:32:45.794120,90.0,1.0
6,2022-02-16 03:42:45.794120,80.0,1.0
7,2022-02-16 03:52:45.794120,70.0,1.0
8,2022-02-16 04:00:05.915053,70.0,2.0
9,2022-02-16 04:02:45.794120,60.0,2.0


---
# done!
thats all the peices we needed for data manage: 
1. save the data in rdbms format
2. incrementally save it
3. pull the data from disk by query so we can pull only the last x hours or something
4. once I pull them all merge them into one dataset, filling in nulls